In [0]:
CREATE OR REFRESH STREAMING TABLE mitrais_databricks.indra_schema.user_drugs_silver
(CONSTRAINT valid_timestamp EXPECT (drug_date_taken > cast(DATE'2024-09-01' AS TIMESTAMP)) ON VIOLATION DROP ROW)
COMMENT "Filtered drugs data"
TBLPROPERTIES ("quality" = "silver")
AS
SELECT
  u.id as user_id,
  u.first_name AS user_first_name,
  u.last_name AS user_last_name,
  u.email_user,
  u.gender,
  u.state_country,
  u.country,
  d.hospital_drug_id,
  d.hospital_npi,
  d.hospital_name,
  d.hospital_city,
  d.drug_name,
  d.drug_brand,
  d.date_taken AS drug_date_taken
FROM 
 (SELECT *
  FROM STREAM(mitrais_databricks.indra_schema..users_bronze_raw)) AS u
INNER JOIN STREAM(mitrais_databricks.indra_schema.hospital_drug_bronze_raw) d ON u.id = d.user_id
--   (SELECT *
--   FROM STREAM(users_bronze_raw)) AS u
-- INNER JOIN STREAM(hospital_drug_bronze_raw) d ON u.user_id = d.drug_user_id
WHERE
  u.email_user IS NOT NULL
  AND u.state_country IS NOT NULL
  AND u.country IS NOT NULL
  AND d.drug_brand IS NOT NULL
  AND d.drug_name IS NOT NULL
  AND d.date_taken IS NOT NULL